<a href="https://colab.research.google.com/github/AstroWLAN/CryptoConditions/blob/main/Crypto_Conditions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Crypto Conditions 🔐**
### **Introduction**
This notebook contains some practical experiments with various types of crypto conditions<br>
A brief introduction to the topic is available in this Github [repo](https://github.com/AstroWLAN/CryptoConditions/tree/main)

In [ ]:
# Environment setup
!pip install cryptoconditions

# Imports modules
import nacl
import secrets
import hashlib
import binascii
import cryptoconditions as cryptocon

# Formats the fingerprint in HEX format and prints the condition
def render_condition(payload,condition_type, derived: bool) :
    fingerprint = payload.get('fingerprint')
    cost = payload.get('cost')
    if derived :
      print('\u001b[1mDERIVED CONDITION\u001b[0m 🧬')
      print(f'\u001b[38;5;246mCondition generated from the fulfillment\u001b[0m','\n')
    else :
      print('\u001b[1mCONDITION\u001b[0m 🔦')
      print(f'\u001b[38;5;246mRequirement that must be met\u001b[0m','\n')
    print(f'{condition_type} \u001b[38;5;38m\u001b[1mCondition\u001b[0m ::=','{')
    print(f'    fingerprint    {fingerprint.hex()}')
    print(f'    cost           {cost}')
    print('}','\n')

# Formats the fingerprint in HEX format and prints the subcondition
def render_subcondition(payload,subcondition_type) :
    fingerprint = payload.get('fingerprint')
    cost = payload.get('cost')
    print(f'{subcondition_type} \u001b[38;5;180m\u001b[1mSubcondition\u001b[0m ::=','{')
    print(f'    fingerprint    {fingerprint.hex()}')
    print(f'    cost           {cost}')
    print('}','\n')

# Prints the content of the fulfillment
def render_fulfillment(payload,fulfillment_type) :
    print('\u001b[1mFULFILLMENT\u001b[0m 📦')
    print(f'\u001b[38;5;246mCryptographic proof that satisfies the condition\u001b[0m','\n')
    print(f'{fulfillment_type} \u001b[38;5;38m\u001b[1mFulfillment\u001b[0m ::=','{')
    match fulfillment_type :
        # Preimage SHA256
        case cryptocon.PreimageSha256.TYPE_ASN1 :
            preimage = payload.get('preimage')
            print(f'    preimage       {preimage.hex()}')
        # ED25519 SHA256
        case cryptocon.Ed25519Sha256.TYPE_ASN1 :
            publicKey = payload.get('publicKey')
            signature = payload.get('signature')
            print(f'    publicKey      {publicKey.hex()}')
            print(f'    signature      {signature.hex()}')
        # Threshold SHA256
        case cryptocon.ThresholdSha256.TYPE_ASN1 :
            subfulfillments = payload.get('subfulfillments')
            subconditions = payload.get('subconditions')
            print(f'    subfulfillments    {subfulfillments}')
            print(f'    subconditions      {subconditions}')
        # Default [ represents an error ]
        case _ :
            print('    payload    \u001b[38;5;174m\u001b[1mbad payload\u001b[0m')
    print('}','\n')

# Prints the content of the subfulfillment
def render_subfulfillment(payload,subfulfillment_type) :
    print(f'{subfulfillment_type} \u001b[38;5;180m\u001b[1mSubfulfillment\u001b[0m ::=','{')
    match subfulfillment_type :
        # Preimage SHA256
        case cryptocon.PreimageSha256.TYPE_ASN1 :
            preimage = payload.get('preimage')
            print(f'    preimage       {preimage.hex()}')
    print('}','\n')

### **Preimage SHA256**
Preimage SHA256 relies on the SHA256 hash function

### **Condition** &nbsp;🔦
The fingerprint of a Preimage SHA256 condition is the SHA256 hash digest of the preimage
> SHA256 hash of the secret message

```
PreimageSHA256 CONDITION ::= {
    fingerprint    SHA256(preimage=secret_message)
    cost           INTEGER
}
```

### **Fulfillment** &nbsp;📦
The fulfillment contains the preimage of the SHA256 cryptographic hash function
> It is the secret message

```
PreimageSHA256 FULFILLMENT ::= {
    preimage    secret_message
}
```

### **Validation** &nbsp;🔑
A Preimage SHA256 fulfillment is considered valid if the fingerprint of the condition matches the SHA256 hash digest of the fulfillment itself

In [ ]:
# Calculates the SHA256 hash digest of a given encoded message
def sha256(message_bytes) :
    hash = hashlib.sha256()
    hash.update(message_bytes)
    return hash.hexdigest()

# Defines the secret message [ the preimage ] and encodes it
secret_message = 'Oppenheimer is a great movie!'
secret_message_encoded = secret_message.encode('utf-8')
secret_message_hashed = sha256(message_bytes=secret_message_encoded)
print(f'\u001b[1mSecret message\u001b[0m\t\t\"{secret_message}\"')
print(f'\u001b[38;5;246mHEX representation\t{secret_message_encoded.hex()}\u001b[0m')
print(f'\u001b[38;5;246mSHA256 hash digest\t{secret_message_hashed}\u001b[0m','\n')

# Builds the desired condition from scratch
condition = cryptocon.Condition()
condition.type_id = cryptocon.PreimageSha256.TYPE_ID
condition.hash = binascii.unhexlify(secret_message_hashed)
condition.cost = len(secret_message)
condition.subtypes = None
condition_payload = condition.to_asn1_dict().get(cryptocon.PreimageSha256.TYPE_ASN1)
render_condition(payload=condition_payload,condition_type=cryptocon.PreimageSha256.TYPE_ASN1,derived=False)

# Creates a fulfillment for the condition
fulfillment = cryptocon.PreimageSha256(preimage=secret_message_encoded)
fulfillment_payload = fulfillment.asn1_dict_payload
render_fulfillment(payload=fulfillment_payload,fulfillment_type=cryptocon.PreimageSha256.TYPE_ASN1)

# Validates the fulfillment
print('\u001b[1mVALIDATION\u001b[0m 🔑 ')
print(f'\u001b[38;5;246mValidates the fulfillment against the condition\u001b[0m','\n')
fulfillment_hash = fulfillment.generate_hash()
condition_fingerprint = condition_payload.get('fingerprint')

if condition_fingerprint.hex() == fulfillment_hash.hex() :
  print(f'\u001b[1mFingerprint\u001b[0m\t\t\t\t\u001b[38;5;246m{condition_fingerprint.hex()}\u001b[0m')
  print(f'\u001b[1mHash digest\u001b[0m of the fulfillment\t\t\u001b[38;5;246m{fulfillment_hash.hex()}\u001b[0m', '\n')
  print('The fulfillment is \u001b[38;5;2m\u001b[1mVALID\u001b[0m')
else :
  print('The fulfillment is \u001b[38;5;174m\u001b[1mNOT VALID\u001b[0m')

# Creates a backup [ needed later on ]
preimageSHA256_condition = condition
preimageSHA256_fulfillment = fulfillment

Secret message		"Oppenheimer is a great movie!"
HEX representation	4f7070656e6865696d65722069732061206772656174206d6f76696521
SHA256 hash digest	51aa687b097ade45ea0cf18c5070362287ff06d626ffed1cf799789d3495915a 

CONDITION 🔦
Requirement that must be met 

preimageSha256 Condition ::= {
    fingerprint    51aa687b097ade45ea0cf18c5070362287ff06d626ffed1cf799789d3495915a
    cost           29
} 

FULFILLMENT 📦
Cryptographic proof that satisfies the condition 

preimageSha256 Fulfillment ::= {
    preimage       4f7070656e6865696d65722069732061206772656174206d6f76696521
} 

VALIDATION 🔑 
Validates the fulfillment against the condition 

Fingerprint				51aa687b097ade45ea0cf18c5070362287ff06d626ffed1cf799789d3495915a
Hash digest of the fulfillment		51aa687b097ade45ea0cf18c5070362287ff06d626ffed1cf799789d3495915a 

The fulfillment is VALID


### **Ed25519 SHA256**
Ed25519SHA256 relies on the SHA256 and SHA512 hash algorithms in combination with the ED25519 signature scheme

### **Condition** &nbsp;🔦
The fingerprint of an Ed25519SHA256 condition is the SHA256 hash value of the DER encoded Ed25519 public key

```
ed25519SHA256 CONDITION ::= {
    fingerprint    SHA256(publicKey.encode)
    cost           INTEGER
}
```

### **Fulfillment** &nbsp;📦
The fulfillment contains the public key for the Ed25519 signature scheme and the signature itself
> The signature is generated using the associated private key

```
ed25519SHA256 FULFILLMENT ::= {
    publicKey   ED25519 publicKey
    signature   ED25519 privateKey.sign(secret_message)
}
```

### **Validation** &nbsp;🔑
An Ed25519SHA256 fulfillment is accepted if both these criteria are met :
* The public key verifies the signature of the secret message
> In other words the signature is valid

* The condition derived from the fulfillment matches the given one


In [ ]:
# Defines the secret message and encodes it
secret_message = 'Oppenheimer is portrayed by Cillian Murphy in this biopic!'
secret_message_encoded = secret_message.encode('utf-8')
print(f'\u001b[1mSecret message\u001b[0m\t\t\"{secret_message}\"','\n')

# Generates a key pair for the ED25519 signing scheme using the provided seed
seed = '8a05a49b44baf8438ff81f69f4bf5c34ed82090482a0f62171ff7cc620460cf4'
private_key = nacl.signing.SigningKey(bytes.fromhex(seed))
public_key = private_key.verify_key
print(f'\u001b[1mPublic Key\u001b[0m \t\t{binascii.hexlify(public_key.encode()).decode()}\u001b[0m')
print(f'\u001b[1mPrivate Key\u001b[0m \t\t{binascii.hexlify(private_key.encode()).decode()}\u001b[0m','\n')

# Builds the desired condition from scratch
condition_URI = 'ni:///sha-256;90C7I9HscZoAu6jv2z27nRHzKPSOIyFIx496vCWlQHk?fpt=ed25519-sha-256&cost=131072'
condition = cryptocon.Condition.from_uri(condition_URI)
condition_payload = condition.to_asn1_dict().get(cryptocon.Ed25519Sha256.TYPE_ASN1)
render_condition(payload=condition_payload,condition_type=cryptocon.Ed25519Sha256.TYPE_ASN1,derived=False)

# Creates a fulfillment for the condition
fulfillment = cryptocon.Ed25519Sha256()
fulfillment.sign(message=secret_message_encoded, private_key=bytes.fromhex(seed))
fulfillment_payload = fulfillment.asn1_dict_payload
render_fulfillment(payload=fulfillment_payload,fulfillment_type=cryptocon.Ed25519Sha256.TYPE_ASN1)

# Validates the fulfillment
print('\u001b[1mVALIDATION\u001b[0m 🔑 ')
print(f'\u001b[38;5;246mValidates the fulfillment against the condition\u001b[0m','\n')
valid_signature = False
valid_derived_condition = False

# Verifies the signature
print(f'\u001b[38;5;246mValidating the signature using the public key...\u001b[0m','\n')
try :
    public_key.verify(secret_message_encoded,fulfillment_payload.get('signature'))
    print('Signature \u001b[38;5;2m\u001b[1mVerified\u001b[0m','\n')
    valid_signature = True
except nacl.exceptions.BadSignatureError:
    print('Signature \u001b[38;5;174m\u001b[1mUnverified\u001b[0m','\n')

# Derives the condition from the fulfillment and compares it to the original one
print(f'\u001b[38;5;246mDeriving the condition from the fulfillment...\u001b[0m','\n')
derived_condition = fulfillment.condition
derived_condition_payload = derived_condition.to_asn1_dict().get(cryptocon.Ed25519Sha256.TYPE_ASN1)
render_condition(payload=derived_condition_payload,condition_type=cryptocon.Ed25519Sha256.TYPE_ASN1,derived=True)
if derived_condition == condition :
  valid_derived_condition = True

if valid_signature and valid_derived_condition :
  print('The fulfillment is \u001b[38;5;2m\u001b[1mVALID\u001b[0m')
else :
  print('The fulfillment is \u001b[38;5;174m\u001b[1mNOT VALID\u001b[0m')

Secret message		"Oppenheimer is portrayed by Cillian Murphy in this biopic!" 

Public Key 		2d331ac731568725ae616fdf0957b44584c4913f4b091333f559ca7386f98331
Private Key 		8a05a49b44baf8438ff81f69f4bf5c34ed82090482a0f62171ff7cc620460cf4 

CONDITION 🔦
Requirement that must be met 

ed25519Sha256 Condition ::= {
    fingerprint    f740bb23d1ec719a00bba8efdb3dbb9d11f328f48e232148c78f7abc25a54079
    cost           131072
} 

FULFILLMENT 📦
Cryptographic proof that satisfies the condition 

ed25519Sha256 Fulfillment ::= {
    publicKey      2d331ac731568725ae616fdf0957b44584c4913f4b091333f559ca7386f98331
    signature      394f69ba2a437cacbf7155fd6065501e8ac42d5a97945e6fb2ff8e92fee138b28c61b8330d89d60fd9dc4dc0cbf5dd01cee019562f6a3ba62a5ba82eb69ec40e
} 

VALIDATION 🔑 
Validates the fulfillment against the condition 

Validating the signature using the public key... 

Signature Verified 

Deriving the condition from the fulfillment... 

DERIVED CONDITION 🧬
Condition generated from the fulfillm

## **Threshold SHA256**
ThresholdSHA256 relies on the SHA256 hash function as well as any other algorithms used in the subconditions

### **Condition** &nbsp;🔦
The fingerprint of an ThresholdSHA256 condition is the SHA256 digest of the DER encoded fingerprint contents
```
ThresholdSHA256 CONDITION ::= {
    fingerprint    SHA256(fingerprint_content.encode)
    cost           INTEGER
}

ThresholdSHA256 FINGERPRINT_CONTENTS ::= {
    threshold      INTEGER
    subconditions  SET of subconditions
}
```

### **Fulfillment** &nbsp;📦
The fulfillment contains the subfulfillments and a list of unfulfilled subconditions
```
ThresholdSHA256 FULFILLMENT ::= {
    subfulfillments   SET of subfulfillments
    subconditions     SET of subconditions
}
```

### **Validation** &nbsp;🔑
A ThresholdSHA256 fulfillment is accepted if both these criteria are met :
* All the subfulfillments are valid
* The condition derived from the fulfillment matches the given one

In [ ]:
# Sets the threshold
threshold = 1
print(f'\u001b[1mThreshold\u001b[0m\t\t\"{threshold}\"','\n')

# Builds the desired condition from scratch
condition_URI = 'ni:///sha-256;m00jtYtoLRqXtxambZZS04YuYmoG12xr7Wfu-SzgNd8?fpt=threshold-sha-256&cost=1053&subtypes=preimage-sha-256'
condition = cryptocon.Condition.from_uri(condition_URI)
condition_payload = condition.to_asn1_dict().get(cryptocon.ThresholdSha256.TYPE_ASN1)
render_condition(payload=condition_payload,condition_type=cryptocon.ThresholdSha256.TYPE_ASN1,derived=False)

# Retrieves the sub fulfillments [ the PreimageSHA256 crypto fulfillment built before ] and their subconditions
sub_fulfillment_PreimageSHA256 = preimageSHA256_fulfillment
sub_fulfillment_PreimageSHA256_payload = sub_fulfillment_PreimageSHA256.asn1_dict_payload
sub_condition_PreimageSHA256 = preimageSHA256_condition
sub_condition_PreimageSHA256_payload = sub_condition_PreimageSHA256.to_asn1_dict().get(cryptocon.PreimageSha256.TYPE_ASN1)
render_subcondition(payload=sub_condition_PreimageSHA256_payload, subcondition_type=cryptocon.PreimageSha256.TYPE_ASN1)

# Creates a fulfillment for the condition
fulfillment = cryptocon.ThresholdSha256()
fulfillment.threshold = threshold
fulfillment.add_subfulfillment(sub_fulfillment_PreimageSHA256)
fulfillment_payload = fulfillment.asn1_dict_payload
render_fulfillment(payload=fulfillment_payload,fulfillment_type=cryptocon.ThresholdSha256.TYPE_ASN1)
render_subfulfillment(payload=sub_fulfillment_PreimageSHA256_payload,subfulfillment_type=cryptocon.PreimageSha256.TYPE_ASN1)

# Validates the fulfillment
print('\u001b[1mVALIDATION\u001b[0m 🔑 ')
print(f'\u001b[38;5;246mValidates the fulfillment against the condition\u001b[0m','\n')
valid_subfulfillments = False
valid_derived_condition = False

# Verifies the subfulfillments
print(f'\u001b[38;5;246mValidating the subfulfillments...\u001b[0m','\n')
derived_subcondition_PreimageSHA256 = sub_fulfillment_PreimageSHA256.condition
if derived_subcondition_PreimageSHA256 == preimageSHA256_condition :
  valid_subfulfillments = True
  print('Subfulfillments \u001b[38;5;2m\u001b[1mVerified\u001b[0m','\n')
else :
  print('Subfulfillments \u001b[38;5;174m\u001b[1mUnverified\u001b[0m','\n')

# Derives the condition from the fulfillment and compares it to the original one
print(f'\u001b[38;5;246mDeriving the condition from the fulfillment...\u001b[0m','\n')
derived_condition = fulfillment.condition
derived_condition_payload = derived_condition.to_asn1_dict().get(cryptocon.ThresholdSha256.TYPE_ASN1)
render_condition(payload=derived_condition_payload,condition_type=cryptocon.ThresholdSha256.TYPE_ASN1,derived=True)
if derived_condition == condition :
  valid_derived_condition = True

if valid_subfulfillments and valid_derived_condition :
  print('The fulfillment is \u001b[38;5;2m\u001b[1mVALID\u001b[0m')
else :
  print('The fulfillment is \u001b[38;5;174m\u001b[1mNOT VALID\u001b[0m')

Threshold		"1" 

CONDITION 🔦
Requirement that must be met 

thresholdSha256 Condition ::= {
    fingerprint    9b4d23b58b682d1a97b716a66d9652d3862e626a06d76c6bed67eef92ce035df
    cost           1053
} 

preimageSha256 Subcondition ::= {
    fingerprint    51aa687b097ade45ea0cf18c5070362287ff06d626ffed1cf799789d3495915a
    cost           29
} 

FULFILLMENT 📦
Cryptographic proof that satisfies the condition 

thresholdSha256 Fulfillment ::= {
    subfulfillments    [{'preimageSha256': {'preimage': b'Oppenheimer is a great movie!'}}]
    subconditions      []
} 

preimageSha256 Subfulfillment ::= {
    preimage       4f7070656e6865696d65722069732061206772656174206d6f76696521
} 

VALIDATION 🔑 
Validates the fulfillment against the condition 

Validating the subfulfillments... 

Subfulfillments Verified 

Deriving the condition from the fulfillment... 

DERIVED CONDITION 🧬
Condition generated from the fulfillment 

thresholdSha256 Condition ::= {
    fingerprint    9b4d23b58b682d1a97b716a